# FastText

In [ ]:
# -- 임의로 데이터 개수 줄임 --
# -- 토큰나이징 하는데 시간이 너무 오래 걸림 --

# counts = None
# # cat_id_over_2000 = list()
# # cat_id_under_2000 = list()
# final_df = pd.DataFrame()

# for cat_id in df['cat_id'].unique():
#     counts = df['cat_id'].value_counts()[df['cat_id'].value_counts().index == cat_id][0]
#     if counts > 2000:
#         tmp_df = df[df['cat_id'] == cat_id].sample(2000)
#     else:
#         tmp_df = df[df['cat_id'] == cat_id]
#     final_df = pd.concat([tmp_df, final_df])

# print(final_df.shape)
# final_df.to_csv('/content/drive/MyDrive/reduced_bungae_df.csv')

In [ ]:
# -- 데이터 개수 한번 더 줄임 --
# -- gpu가 감당하지 못하는 데이터 양인 것 같아서 거의 1/10 수준으로 줄임 --
# counts = None
# # cat_id_over_2000 = list()
# # cat_id_under_2000 = list()
# final_df = pd.DataFrame()

# for cat_id in df['cat_id'].unique():
#     counts = df['cat_id'].value_counts()[df['cat_id'].value_counts().index == cat_id][0]
#     if counts > 600:
#         tmp_df = df[df['cat_id'] == cat_id].sample(600)
#     else:
#         tmp_df = df[df['cat_id'] == cat_id]
#     final_df = pd.concat([tmp_df, final_df], ignore_index=True)

# print(final_df.shape)
# final_df.to_csv('/content/drive/MyDrive/reduced_twice_bungae_df.csv')

In [ ]:
# -- google drive에 연결 --
# from google.colab import drive
# drive.mount('/content/drive')

# # -- mecab 설치 --
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
!pip3 install mecab-python3
!pip install konlpy
!pip install transformers
!pip install apex

In [68]:
# -- 필요한 함수들 --
import torch
import torch.nn as nn
import torchtext
from torch.utils.data import Dataset, DataLoader
from konlpy.tag import Mecab

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torchtext.vocab import build_vocab_from_iterator, Vocab

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import string

def bring_and_preprocess_df():
    # -- 상대적으로 개수가 적은 csv 파일 사용 -- 
    df = pd.read_csv('/content/drive/MyDrive/reduced_twice_bungae_df.csv', encoding='utf-8-sig')
    df['cat_id'] = df['cat_id'].astype(int).astype(str)

    # -- 패션과 관련 없는 카테고리 제거 --
    idx_list = list()
    for idx in range(len(df)):
        if (len(df.loc[idx, 'cat_id']) == 3) | (df.loc[idx, 'cat_id'][0] not in ['3', '4']): 
            idx_list.append(idx)

    df = df[~df.index.isin(idx_list)]
    # -- 카테고리당 상품이 1~2개 있을 경우 train_test_split 했을 때 갯수가 너무 적어 안됨, 이후에 수정 필요 --
    df = df[~df['cat_id'].isin(['310060999', '310220999', '320180500'])]
    
    df = df.dropna(axis=0)
    df = df.reset_index(drop=True)
    return df

class CustomDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]
        return text, label

def collate_fn(batch):
    texts, labels = zip(*batch)
    indexed_texts = [torch.LongTensor([word2index.get(token, 0) for token in text]) for text in texts]
    padded_texts = torch.nn.utils.rnn.pad_sequence(indexed_texts, batch_first=True)
    padded_labels = torch.nn.utils.rnn.pad_sequence([torch.tensor(label) for label in labels], batch_first=True)
    return padded_texts, padded_labels

# def collate_fn(batch):
#     texts, labels = zip(*batch)
#     text_lengths = [len(text) for text in texts]
#     max_length = max(text_lengths)

#     texts = [[word2index.get(token, 0) for token in tokenizer(text)] for text in texts]
#     indexed_texts = [torch.LongTensor(text) for text in texts]
#     labels = torch.LongTensor(labels)
#     return indexed_texts, labels

def remove_punct(text):
    """
    문장기호 없애는 함수
    """
    remove_punct_dict = dict((ord(punct), ' ') for punct in string.punctuation)
    text = text.lower().translate(remove_punct_dict)
    return text


def tokenizer(text):
    mecab = Mecab()
    tokens = mecab.morphs(text)
    return tokens


In [53]:
# -- data --
df = bring_and_preprocess_df()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100169 entries, 0 to 100168
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0.1  100169 non-null  int64  
 1   Unnamed: 0    100169 non-null  int64  
 2   product_id    100169 non-null  int64  
 3   product_name  100169 non-null  object 
 4   image_url     100169 non-null  object 
 5   image_cnt     100169 non-null  float64
 6   cat_id        100169 non-null  object 
dtypes: float64(1), int64(3), object(3)
memory usage: 5.3+ MB


In [54]:
# -- preprocessing --

# -- label encoding --
# encoder = LabelEncoder()
# df['label'] = encoder.fit_transform(df['cat_id']) # 이후에 매칭 다시 필요 

# -- removing unnecessary texts --
df['product_name'] = df['product_name'].apply(lambda x : remove_punct(x))

In [55]:
# Load and preprocess your own data
texts = list(df['product_name'])
labels = df['cat_id'].values # 뒤에서 label encoder 다시 사용

# Instantiate tokenizer
# tokenizer = Mecab()

# word2index
def get_word2index(texts, max_vocab_size=None):
    def yield_tokens(texts):
        for text in texts:
            yield tokenizer(text)
    
    vocab = build_vocab_from_iterator(yield_tokens(texts))
    word2index = vocab.get_stoi()

    if max_vocab_size is not None:
        word2index = {k: v for k, v in word2index.items() if v < max_vocab_size}
    
    return word2index

max_vocab_size = 10000 # 바꿔야 될수도?
word2index = get_word2index(texts, max_vocab_size)


In [56]:
# train and test dataset
texts_train, texts_test, labels_train, labels_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

texts_train = [tokenizer(text) for text in texts_train]
texts_test = [tokenizer(text) for text in texts_test]

# Encode labels
label_encoder = LabelEncoder()
labels_train = label_encoder.fit_transform(labels_train)
labels_test = label_encoder.transform(labels_test) # 여기서 label_train에 없는 클래스가 있는 경우 오류가 남. 


In [70]:
# train 
batch_size = 32
train_dataset = CustomDataset(texts_train, labels_train)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
# test
test_dataset = CustomDataset(texts_test, labels_test)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=False)

In [66]:
# pretrained embeddings 
embedding_file = '/content/drive/MyDrive/wiki.ko.vec'
pretrained_embeddings = torchtext.vocab.Vectors(embedding_file)

# vocab = Vocab(word2index, vectors=pretrained_embeddings)

# parameter
vocab_size = len(word2index)
num_classes = 170

embedding_dim = pretrained_embeddings.dim
embedding = nn.Embedding.from_pretrained(pretrained_embeddings.vectors, padding_idx=0)


class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes, embeddings):
        super(FastText, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight = nn.Parameter(embeddings, requires_grad=False)
        self.fc = nn.Linear(embedding_dim, num_classes)

    def forward(self, inputs):
        embedded = self.embedding(inputs).permute(1, 0, 2)
        pooled = torch.mean(embedded, dim=1)
        output = self.fc(pooled)
        return output

# -- 계속 int is not callable 문제가 떴을 때 해결 방법 --
embedding = torch.FloatTensor(pretrained_embeddings.vectors)

model = FastText(vocab_size, embedding_dim, num_classes, embedding)

In [71]:
# -- training --
from sklearn.metrics import accuracy_score

learning_rate = 0.001

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model, optimizer = amp.initialize(model, optimizer, opt_level='O1')
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
num_epochs = 10
accuracy_history = []
for epoch in range(num_epochs):
    model.train()
    for batch_inputs, batch_labels in train_dataloader:

        batch_size = batch_inputs.size(0)  # Get the actual batch size

        if batch_size != batch_inputs.size(0):
            continue  

        batch_inputs = batch_inputs.to(device)
        batch_labels = batch_labels.to(device)

        optimizer.zero_grad()
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()

        # calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        acc = (predicted == batch_labels).sum().item() / batch_size
        print(f"Accuracy: {acc}")

    # Evaluation
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for batch_inputs, batch_labels in test_dataloader:
            # move data to GPU
            batch_inputs = batch_inputs.to(device)
            batch_labels = batch_labels.to(device)

            outputs = model(batch_inputs)
            _, predicted = torch.max(outputs.data, dim=1)
            total += batch_labels.size(0)
            correct += (predicted == batch_labels).sum().item()
        
        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Test Accuracy: {accuracy:.2f}%")

   


IndexError: ignored